<a href="https://colab.research.google.com/github/Jinwooxxi/kaggle_survey2021/blob/main/kaggle_survey_2021_JWK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c kaggle-survey-2021
!unzip kaggle_survey_2021_responses.csv.zip

In [ ]:
# !pip install --upgrade --user matplotlib

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

In [ ]:
response = pd.read_csv('kaggle_survey_2021_responses.csv')
response

In [ ]:
response.drop(index=0, inplace=True) # 첫째 행 삭제, column title은 추후 수정 예정
response.index = range(len(response)) # index 재조정
response = response.fillna('') # NaN → 빈값으로 변경
print(response.shape)
response.head()

In [ ]:
def total_answer(column):
  # 해당 컬럼명이 들어간 컬럼 추출
  col = response.columns[response.columns.str.contains(column)]
  tmp = response[col]

  # 컬럼별 응답 join
  total = tmp.apply(','.join, axis=1)

  # 정규표현식으로 필요없는 콤마 삭제
  import re
  ls = []
  for i in total:
    ls.append(re.sub(',{2,}', ',', i).strip(','))

  # 나눠진 응답 삭제
  response.drop(col, axis=1, inplace=True)
  # 통합 응답 추가
  response[column] = ls

In [ ]:
for i in range(7, 43):
  num = 'Q'+str(i) 
  total_answer(num) 

In [ ]:
response

In [ ]:
response.describe().T

## Q5 → 직군 나누기 (DA, DS, DE, OTHER)

In [ ]:
response['Q5'] = response['Q5'].replace(['Business Analyst', 'Data Analyst', 'Product Manager', 'Program/Project Manager', 'Statistician'],'DA')
response['Q5'] = response['Q5'].replace(['Data Engineer', 'DBA/Database Engineer', 'Machine Learning Engineer', 'Software Engineer', 'Developer Relations/Advocacy'], 'DE')
response['Q5'] = response['Q5'].replace(['Data Scientist', 'Research Scientist'], 'DS')
response['Q5'] = response['Q5'].replace(['Student', 'Currently not employed', 'Other'], 'OTHER')

In [ ]:
response['Q5'].value_counts()

* Q6 코딩, 프로그래밍 사용 연수 
* Q7 프로그래밍 사용 언어
* Q9 주 사용 IDE
* Q10 주 사용 노트북
* Q14 직군별 주 사용 시각화
* Q15 M/L 사용 년수
* Q20 현재 직업 군 or 최근 직업 군
* Q21 회사 규모
* Q25 연봉
* Q32, 33 빅데이터 관련 사용 제품 및 최다 빈도 제품


## Q6 코딩, 프로그래밍 사용 연수

In [ ]:
plt.subplots(figsize=(20,15))
job = response['Q5'].unique()
length = len(job)
col_color = {'< 1 years':'r',
             'I have never written code':'#273c75',
             '1-3 years':'#273c75',
             '3-5 years':'#273c75',
             '5-10 years':'#273c75',
             '10-20 years':'#273c75',
             '20+ years':'#273c75'}

for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q5'] == i]['Q6'].value_counts(ascending=True)
  data.plot(kind='barh', color=pd.DataFrame(data).reset_index()['index'].replace(col_color))
  plt.title(i, size=20)

In [ ]:
response['Q6'].value_counts()

In [ ]:
plt.subplots(figsize=(20,15))
exp = response['Q6'].unique()
length = len(exp)
col_color = {'DA':'r',
             'DE':'#273c75',
             'DS':'#273c75',
             'OTHER':'#273c75'}

for i, j in itertools.zip_longest(exp, range(length)):
  plt.subplot((length/2), 3, j+1)
  data = response[response['Q6'] == i]['Q5'].value_counts(ascending=True)
  data.plot(kind='barh', color=pd.DataFrame(data).reset_index()['index'].replace(col_color))
  plt.title(i, size=20)

## Q7 프로그래밍 사용 언어
* Python 
* R
* SQL 
* C
* C++
* Java
* Javascript 
* Julia
* Swift
* Bash
* MATLAB 
* None
* Other

In [ ]:
response['Q7'] = response['Q7'].str.replace('C\++', '++').str.replace('Javascript', 'JS')

In [ ]:
lang = ['Python', 'R', 'SQL', 'C', '\++', 'Java', 'JS', 'Julia', 'Swift', 'Bash', 'MATLAB', 'None', 'Other']

In [ ]:
plt.subplots(figsize=(20,17))
job = response['Q5'].unique()
length = len(job)

for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q5'] == i]  
  
  lan = []
  cnt = []
  for k in lang:
    lan.append(k)
    cnt.append(len(data[data['Q7'].str.contains(k)]))
  
  df = pd.DataFrame(columns=["language", 'cnt'])
  df['language'] = lan
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df, y='language', x='cnt')
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')

In [ ]:
plt.subplots(figsize=(20,17))
exp = response['Q6'].unique()
exp = np.delete(exp, 6) # 'I have never written code' 코딩경험이 없기 때문에 삭제
length = len(exp)

for i, j in itertools.zip_longest(exp, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q6'] == i]  
  
  lan = []
  cnt = []
  for k in lang:
    lan.append(k)
    cnt.append(len(data[data['Q7'].str.contains(k)]))
  
  df = pd.DataFrame(columns=["language", 'cnt'])
  df['language'] = lan
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df, y='language', x='cnt')
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')

## Q9 주 사용 IDE

In [ ]:
idle = ['JupyterLab', 'RStudio', 'Visual Studio', 'VSCode', 'PyCharm', 'Spyder', 'Notepad',
        'Sublime Text', 'Vim, Emacs, or similar', 'MATLAB', 'Jupyter Notebook', 'None', 'Other']

In [ ]:
plt.subplots(figsize=(20,17))
job = response['Q5'].unique()
length = len(job)

for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q5'] == i]  
  
  ide = []
  cnt = []
  for k in idle:
    ide.append(k)
    cnt.append(len(data[data['Q9'].str.contains(k)]))
  
  df = pd.DataFrame(columns=["ide", 'cnt'])
  df['ide'] = ide
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df, y='ide', x='cnt')
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')

In [ ]:
plt.subplots(figsize=(20,17))
exp = response['Q6'].unique()
exp = np.delete(exp, 6) # 'I have never written code' 코딩경험이 없기 때문에 삭제
length = len(exp)

for i, j in itertools.zip_longest(exp, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q6'] == i]  
  
  ide = []
  cnt = []
  for k in idle:
    ide.append(k)
    cnt.append(len(data[data['Q9'].str.contains(k)]))
  
  df = pd.DataFrame(columns=["ide", 'cnt'])
  df['ide'] = ide
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df, y='ide', x='cnt')
  plt.title(i, size=20)
  plt.subplots_adjust(wspace=0.2, hspace=0.5)
  plt.xlabel('')
  plt.ylabel('')

## Q14 시각화 툴

In [ ]:
visual = ['Matplotlib', 'Seaborn', 'Plotly', 'ggplot', 'Shiny', 'D3 js', 'Altair', 'Bokeh', 'Geoplotlib',
       'Leaflet', 'None', 'Other']

In [ ]:
plt.subplots(figsize=(20,17))
job = response['Q5'].unique()
length = len(job)

for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q5'] == i]  
  
  vis = []
  cnt = []
  for k in visual:
    vis.append(k)
    cnt.append(len(data[data['Q14'].str.contains(k)]))
  
  df = pd.DataFrame(columns=["vis", 'cnt'])
  df['vis'] = vis
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df, y='vis', x='cnt')
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')

In [ ]:
plt.subplots(figsize=(20,17))
exp = response['Q6'].unique()
exp = np.delete(exp, 6) # 'I have never written code' 코딩경험이 없기 때문에 삭제
length = len(exp)

for i, j in itertools.zip_longest(exp, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q6'] == i]  
  
  vis = []
  cnt = []
  for k in visual:
    vis.append(k)
    cnt.append(len(data[data['Q14'].str.contains(k)]))
  
  df = pd.DataFrame(columns=["vis", 'cnt'])
  df['vis'] = vis
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df, y='vis', x='cnt')
  plt.title(i, size=20)
  plt.subplots_adjust(wspace=0.2, hspace=0.5)
  plt.xlabel('')
  plt.ylabel('')

## Q15 M/L 사용 년수

In [ ]:
response['Q15'].unique()

In [ ]:
plt.subplots(figsize=(20,15))
job = response['Q5'].unique()
length = len(job)
col_color = {'Under 1 year':'r',
             'I do not use machine learning methods':'#273c75',
             '1-2 years':'#273c75',
             '2-3 years':'#273c75',
             '3-4 years':'#273c75',
             '4-5 years':'#273c75',
             '5-10 years':'#273c75',
             '10-20 years':'#273c75',
             '20 or more years':'#273c75',
             '':'#273c75'}

for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q5'] == i]['Q15'].value_counts(ascending=True)
  data.plot(kind='barh', color=pd.DataFrame(data).reset_index()['index'].replace(col_color))
  plt.title(i, size=20)

In [ ]:
plt.subplots(figsize=(20,20))
exp = response['Q15'].unique()
length = len(exp)
col_color = {'DA':'r',
             'DE':'#273c75',
             'DS':'#273c75',
             'OTHER':'#273c75'}

for i, j in itertools.zip_longest(exp, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q15'] == i]['Q5'].value_counts(ascending=True)
  data.plot(kind='barh', color=pd.DataFrame(data).reset_index()['index'].replace(col_color))
  plt.title(i, size=20)

# Q20 현재 종사 산업군 (or 과거 산업군)

In [ ]:
sorted(response['Q20'].unique())

In [ ]:
sorted(response['Q20'].unique())[1:]

In [ ]:
response['Q20'].value_counts()

In [ ]:
plt.subplots(figsize=(20,17))
job = response['Q5'].unique()
length = len(job)
job_uni = sorted(response['Q20'].unique())[1:] # 빈 공백 제거

for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q5'] == i]  
  
  jobs = []
  cnt = []
  for k in job_uni:
    jobs.append(k)
    cnt.append(len(data[data['Q20']==k]))
  
  df = pd.DataFrame(columns=["jobs", 'cnt'])
  df['jobs'] = jobs
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df[:10], y='jobs', x='cnt')
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')

In [ ]:
plt.subplots(figsize=(20,17))
job = response['Q6'].unique()
length = len(job)
job_uni = sorted(response['Q20'].unique())[1:] # 빈 공백 제거

for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 3, j+1)
  data = response[response['Q6'] == i]  
  
  jobs = []
  cnt = []
  for k in job_uni:
    jobs.append(k)
    cnt.append(len(data[data['Q20']==k]))
  
  df = pd.DataFrame(columns=["jobs", 'cnt'])
  df['jobs'] = jobs
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df[:10], y='jobs', x='cnt')
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')

# Q21 회사 규모

In [ ]:
sorted(response['Q21'].unique())

In [ ]:
response['Q21'].value_counts()

In [ ]:
plt.subplots(figsize=(20,17))
job = response['Q5'].unique()
length = len(job)
sizes = sorted(response['Q21'].unique())[1:]


for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q5'] == i]  
  
  size = []
  cnt = []
  for k in sizes:
    size.append(k)
    cnt.append(len(data[data['Q21']==k]))
  
  df = pd.DataFrame(columns=["size", 'cnt'])
  df['size'] = size
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df, y='size', x='cnt')
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')

In [ ]:
plt.subplots(figsize=(20,17))
job = response['Q6'].unique()
length = len(job)
sizes = sorted(response['Q21'].unique())[1:]


for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 3, j+1)
  data = response[response['Q6'] == i]  
  
  size = []
  cnt = []
  for k in sizes:
    size.append(k)
    cnt.append(len(data[data['Q21']==k]))
  
  df = pd.DataFrame(columns=["size", 'cnt'])
  df['size'] = size
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df, y='size', x='cnt')
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')

# Q24 맡은 업무 중 데이터와 관련된 내용

In [ ]:
lst = ['Analyze and understand data to influence product or business decisions',
       'Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data',
       'Build prototypes to explore applying machine learning to new areas',
       'Build and/or run a machine learning service that operationally improves my product or workflows',
       'Experimentation and iteration to improve existing ML models',
       'Do research that advances the state of the art of machine learning',
       'None of these activities are an important part of my role at work',
       'Other']

In [ ]:
plt.subplots(figsize=(20,17))
job = response['Q5'].unique()
length = len(job)

for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q5'] == i]  
  
  ls = []
  cnt = []
  for k in lst:
    ls.append(k)
    cnt.append(len(data[data['Q24'].str.contains(k)]))
  
  df = pd.DataFrame(columns=["ls", 'cnt'])
  df['ls'] = ls
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df, y='ls', x='cnt')
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')

# Q25 연봉

In [ ]:
sorted(response['Q25'].unique())

In [ ]:
response['Q25'].value_counts()

In [ ]:
plt.subplots(figsize=(20,17))
job = response['Q5'].unique()
length = len(job)
compensation = sorted(response['Q25'].unique())[1:]


for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q5'] == i]  
  
  income = []
  cnt = []
  for k in compensation:
    income.append(k)
    cnt.append(len(data[data['Q25']==k]))
  
  df = pd.DataFrame(columns=["income", 'cnt'])
  df['income'] = income
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df, y='income', x='cnt')
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')

In [ ]:
plt.subplots(figsize=(20,17))
job = response['Q6'].unique()
length = len(job)
compensation = sorted(response['Q25'].unique())[1:]


for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 3, j+1)
  data = response[response['Q6'] == i]  
  
  income = []
  cnt = []
  for k in compensation:
    income.append(k)
    cnt.append(len(data[data['Q25']==k]))
  
  df = pd.DataFrame(columns=["income", 'cnt'])
  df['income'] = income
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df, y='income', x='cnt')
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')